In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy as sp

In [2]:
pwd

'/Users/shayneufeld/GitHub/mouse_bandit/jupyter_notebooks'

In [4]:
import sys
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit/data_preprocessing_code')
import bandit_preprocessing as bp

In [5]:
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given']

In [5]:
df_90 = pd.read_csv('/Users/shayneufeld/GitHub/ac209a_project/data/trials_90_10.csv',names=columns)
df_70 = pd.read_csv('/Users/shayneufeld/GitHub/ac209a_project/data/trials_70_30.csv',names=columns)

In [21]:
df_90.head(10)

,Elapsed Time (s),Since last trial (s),Trial Duration (s),Port Poked,Right Reward Prob,Left Reward Prob,Reward Given
0,0.891,0.000,0.891,Right,0.9,0.1,1
1,6.640,1.523,0.540,Right,0.9,0.1,1
2,8.708,1.714,0.354,Right,0.9,0.1,1
3,11.230,2.035,0.487,Right,0.9,0.1,1
4,13.759,2.090,0.439,Right,0.9,0.1,1
5,16.712,2.380,0.573,Right,0.9,0.1,1
6,19.226,2.240,0.274,Right,0.9,0.1,1
7,21.624,2.083,0.315,Right,0.9,0.1,1
8,33.672,3.898,0.582,Left,0.9,0.1,0
9,46.415,10.666,0.688,Right,0.9,0.1,1


In [7]:
df_90.loc[df_90['Port Poked'] == 1,'Port Poked'] = 'Right'
df_90.loc[df_90['Port Poked'] == 2,'Port Poked'] = 'Left'

In [19]:
feature_names

['Elapsed Time (s)',
 'Since last trial (s)',
 'Trial Duration (s)',
 'Port Poked',
 'Right Reward Prob',
 'Left Reward Prob',
 'Reward Given']

# Important Note:

For now, will only consider 10th trial and onwards for simplicity. 
**May want to revisit this for model that builds its own features for simulating behavior session**

In [10]:
n_trials = df_90.shape[0]
n_trials

n_summary = 10;
n_indi = 5;

In [30]:
df = df_70.copy()
n_trials = df.shape[0]
n_trials

n_summary = 10;
n_indi = 5;

num_cols = 5 + 4*n_indi + 2 + 1 #5 summary rows, 4 cols for each past trial, 2 for current trial + 1 decision!
feature_matrix = np.zeros((n_trials-n_summary,num_cols))

In [32]:
np.sum(past_trials['Port Poked'] == 'Left')

TypeError: invalid type comparison

In [33]:
past_trials

,Elapsed Time (s),Since last trial (s),Trial Duration (s),Port Poked,Right Reward Prob,Left Reward Prob,Reward Given
0,22.024,1.478,0.346,2,0.3,0.7,1
1,24.812,2.287,0.501,2,0.3,0.7,0
2,31.390,5.923,0.655,2,0.3,0.7,1
3,33.180,1.441,0.349,2,0.3,0.7,0
4,35.288,1.670,0.438,1,0.3,0.7,0
5,38.371,1.874,0.323,2,0.3,0.7,1
6,40.862,2.184,0.307,2,0.3,0.7,1
7,43.383,2.248,0.273,2,0.3,0.7,1
8,47.129,3.422,0.324,2,0.3,0.7,1
9,50.002,2.610,0.263,2,0.3,0.7,0


In [36]:
for j,i in enumerate(np.arange(n_summary,n_trials)):
    
    #extract the 'n_summary' trials we need to consider. Assume that n_summary > n_indi
    past_trials = df.iloc[i-n_summary:i]
    
    #calculate summary statistics
    
    #left and right ports
    feature_matrix[j,0] = np.sum(past_trials['Port Poked'] == 2)
    feature_matrix[j,1] = np.sum(past_trials['Port Poked'] == 1)
    
    #left and right rewardsd
    feature_matrix[j,2] = np.sum(past_trials.loc[past_trials['Port Poked']== 2,'Reward Given'])
    feature_matrix[j,3] = np.sum(past_trials.loc[past_trials['Port Poked']== 1,'Reward Given'])
    
    #streak
    '''
    approach: take the derivative of the reward boolean. Streak is number of [0s + 1] (from the end). 
    the valence of the streak is the sign of first non-zero entry (from the end)
    '''
    streak_vec = np.flipud(np.diff(past_trials['Reward Given'].values)) #reverse order of array so end is
    #at the front. This makes it easier to find the first non-zero entry
    streak_len = np.nonzero(streak_vec)[0]
    
    if len(streak_len) == 0: #have to deal with case where streak is all 10 previous trials!
        streak_len = 10
        if np.sum(past_trials['Reward Given'].values > 0):
            streak_sign = 1
        else:
            streak_sign = -1
    #otherwise, streak is less then 10 trials and things are simpler.
    else:
        streak_len = streak_len[0]
        streak_sign = streak_vec[streak_len]
    
        feature_matrix[j,4] = (streak_len+1)*streak_sign
    
    '''
    INDIVIDUAL TRIALS
    '''
    k = 0
    for icol,itrial in enumerate(np.arange(n_indi,0,-1)):
        
        past_trial = past_trials.iloc[-itrial,:]
        
        #which port
        if past_trial['Port Poked'] == 1:
            feature_matrix[j,5+k] = 0
        elif past_trial['Port Poked'] == 2:
            feature_matrix[j,5+k] = 1
        else:
            print('Error port not Left or Right')
        k += 1
        
        #reward given:
        feature_matrix[j,5+k] = past_trial['Reward Given']
        k += 1
        
        #ITI
        feature_matrix[j,5+k] = past_trial['Since last trial (s)']
        k += 1
        
        #trial time
        feature_matrix[j,5+k] = past_trial['Trial Duration (s)']
        k += 1
    
    '''
    CURRENT TRIAL
    '''
    current_trial = df.iloc[i,:]
    feature_matrix[j,5+k] = current_trial['Since last trial (s)']
    k += 1
    feature_matrix[j,5+k] = current_trial['Trial Duration (s)']
    k += 1
    '''
    DECISION
    '''
    if current_trial['Port Poked'] == 1:
        feature_matrix[j,5+k] = 0
    elif current_trial['Port Poked'] == 2:
        feature_matrix[j,5+k] = 1
    else:
        print('Error decision port not Left or Right')

In [22]:
'''
convert into feature dataframe
'''
feature_names = ['Previous_10_Left_Choice',
                 'Previous_10_Right_Choice',
                 'Previous_10_Left_Reward',
                 'Previous_10_Right_Reward',
                 'Streak',
                 '5_Port',
                 '5_Reward',
                 '5_ITI',
                 '5_trialDuration',
                 '4_Port',
                 '4_Reward',
                 '4_ITI',
                 '4_trialDuration',
                 '3_Port',
                 '3_Reward',
                 '3_ITI',
                 '3_trialDuration',
                 '2_Port',
                 '2_Reward',
                 '2_ITI',
                 '2_trialDuration',
                 '1_Port',
                 '1_Reward',
                 '1_ITI',
                 '1_trialDuration',
                 '0_ITI',
                 '0_trialDuration',
                 'Decision'
                 ]

In [23]:
feature_df = pd.DataFrame(data=feature_matrix,index=None,columns=feature_names)
feature_df.head(5)

,Previous_10_Left_Choice,Previous_10_Right_Choice,Previous_10_Left_Reward,Previous_10_Right_Reward,Streak,5_Port,5_Reward,5_ITI,5_trialDuration,4_Port,...,2_Reward,2_ITI,2_trialDuration,1_Port,1_Reward,1_ITI,1_trialDuration,0_ITI,0_trialDuration,Decision
0,1.0,9.0,0.0,9.0,1.0,0.0,1.0,2.380,0.573,0.0,...,0.0,3.898,0.582,0.0,1.0,10.666,0.688,1.650,0.428,1.0
1,2.0,8.0,0.0,8.0,-1.0,0.0,1.0,2.240,0.274,0.0,...,1.0,10.666,0.688,1.0,0.0,1.650,0.428,1.421,0.481,0.0
2,2.0,8.0,0.0,8.0,1.0,0.0,1.0,2.083,0.315,1.0,...,0.0,1.650,0.428,0.0,1.0,1.421,0.481,2.850,0.373,0.0
3,2.0,8.0,0.0,8.0,2.0,1.0,0.0,3.898,0.582,0.0,...,1.0,1.421,0.481,0.0,1.0,2.850,0.373,3.563,0.394,0.0
4,2.0,8.0,0.0,8.0,3.0,0.0,1.0,10.666,0.688,1.0,...,1.0,2.850,0.373,0.0,1.0,3.563,0.394,2.821,0.390,0.0
